<h2>Problem 1</h2>

In [2]:
import numpy as np
def expectation_maximization(clusters, num_iterations, data, initial_mu, initial_sigma, initial_p, should_print):
    """ Performs expectation maximiazation the data and prints out the mean, standard deviation, mixing 
    probabilities and posterior probabilities at each step.
    
    Params:
    clusters: number of clusters
    num_iterations: number of iteraions to run EM for
    data: data on which EM is run
    initial_mu: initial mean
    initial_sigma: initial standard deviation
    initial_p: initital mixing probability
    should_print: indicates if we should print the paremeters at each step
    
    Returns:
    Calculated mean, standard deviation and mixinf probability 
    """
    k = clusters
    mu = initial_mu
    sigma = initial_sigma
    p = initial_p
    
    N, D = data.shape
    for iteration in range(0, num_iterations):
        prob = []
        for i in range (0, N):
            prob.append(expectation(p, mu, sigma, data[i], k, D))
        prob = np.array(prob)
        mu, sigma, p = maximization(prob, data, k)
        if(should_print):
            print("***** iteration {} ******".format(iteration))
            print("mean: {}".format( mu))
            print("standar deviation: {}".format( sigma))
            print("mixing probability: {}".format(p))
            print("posterior probability{}".format(np.array(prob)))
            print("\n")
    
    return mu, sigma, p

def expectation(p, mu, sigma, row, k, D):    
    sum = 0
    for i in range(0, k):
        sum+=p[i]*g(mu[i], sigma[i], row, D)
        
    prob = []
    for i in range(0, k):
        prob.append((p[i]*g(mu[i], sigma[i], row, D))/sum)
    return np.array(prob)

def maximization(prob, data, k):
    N, D = data.shape
    # Maximixe mean
    mean_top = []
    for cluster in range(0, k):
        sum = 0
        for i in range(0, N):
            sum+=(prob.transpose()[cluster][i] * data[i])
        mean_top.append(sum)
    mean_top = np.array(mean_top)
    
    mean_bottom = []
    for cluster in range(0, k):
        sum = 0
        for i in range(0, N):
            sum+=prob.transpose()[cluster][i]
        mean_bottom.append(sum)
    mean_bottom = np.array(mean_bottom)
    mu = mean_top/mean_bottom.reshape(-1,1)
    
    #Maximize sigma
    sigma_top = []
    for cluster in range(0, k):
        sum = 0
        for i in range(0, N):
            inter = data[i] - mu[cluster]
            sum+=(prob.transpose()[cluster][i] * inter.dot(inter))
        sigma_top.append(sum)
    sigma_top = np.array(sigma_top)
    sigma = np.sqrt(sigma_top/(2 * mean_bottom))
    
    # Maximize mixing probabilites 
    p = mean_bottom/N
    
    return mu, sigma, p
    
        

def g(mu, sigma, row, D):
    first = 1/np.power(np.sqrt(2 * np.pi )* sigma, D)
    inter = (row-mu)/sigma
    second = np.exp(-0.5* inter.dot(inter))
    calc =  first * second 
    return calc
    
    

In [3]:
data = [[1, 2],[4, 2],[1, 3],[4, 3]]
np.array(data).std(axis =0)

array([1.5, 0.5])

In [4]:
# Use the initial variables provided in the lecture slides.
mu = np.array([[2.1766, 2.3922],[3.7571, 2.9190]])
sigma = np.array([1.1547, 1.1547])
p = np.array([0.5, 0.5])
mu, sigma, p = expectation_maximization(2, 5, np.array(data), mu, sigma, p, True)

***** iteration 0 ******
mean: [[1.62322012 2.47791236]
 [3.69837745 2.53018925]]
standar deviation: [0.9302605  0.72903364]
mixing probability: [0.57748752 0.42251248]
posterior probability[[0.93024668 0.06975332]
 [0.27574969 0.72425031]
 [0.89983426 0.10016574]
 [0.20411943 0.79588057]]


***** iteration 1 ******
mean: [[1.10697853 2.49926496]
 [3.99551753 2.50078913]]
standar deviation: [0.52890956 0.36292334]
mixing probability: [0.51774185 0.48225815]
posterior probability[[0.99861806 0.00138194]
 [0.03838789 0.96161211]
 [0.99849966 0.00150034]
 [0.03546179 0.96453821]]


***** iteration 2 ******
mean: [[1.0000008 2.5      ]
 [4.        2.5      ]]
standar deviation: [0.35355508 0.35355339]
mixing probability: [0.50000013 0.49999987]
posterior probability[[1.00000000e+00 1.95794875e-15]
 [2.67286455e-07 9.99999733e-01]
 [1.00000000e+00 1.97489685e-15]
 [2.64992665e-07 9.99999735e-01]]


***** iteration 3 ******
mean: [[1.  2.5]
 [4.  2.5]]
standar deviation: [0.35355339 0.353553

<h2> Problem 2 </h2>

In [5]:
import seaborn as sns
iris = sns.load_dataset("iris")

In [6]:
iris_data_cleaned = iris.drop(columns=['species']).values

In [7]:
calculated_mu = iris_data_cleaned.mean(axis=0)

In [8]:
calculated_sigma  = iris_data_cleaned.std(axis=0)

In [9]:
clusters = 3
mu = (calculated_mu.transpose().reshape(-1,1) * np.ones(clusters)) + (calculated_sigma.reshape(-1,1) * np.random.randn(clusters))
mu = mu.transpose()
sigma = (calculated_sigma.sum()/len(calculated_sigma)) *np.ones(clusters)
p = np.ones(clusters)/clusters
mu, sigma, p = expectation_maximization(clusters, 100, iris_data_cleaned, mu, sigma, p, False)

In [10]:
mu

array([[5.92337106, 2.75769455, 4.43209925, 1.45007568],
       [5.00600096, 3.42798938, 1.46201753, 0.24600965],
       [6.82873748, 3.06330432, 5.69913517, 2.05410915]])

In [11]:
sigma

array([0.59719306, 0.38926979, 0.60782215])

In [12]:
p

array([0.41731006, 0.33333706, 0.24935288])

In [13]:
iris[iris['species']=='setosa'].mean()

sepal_length    5.006
sepal_width     3.428
petal_length    1.462
petal_width     0.246
dtype: float64

In [14]:
iris[iris['species']=='virginica'].mean()

sepal_length    6.588
sepal_width     2.974
petal_length    5.552
petal_width     2.026
dtype: float64

In [15]:
iris[iris['species']=='versicolor'].mean()

sepal_length    5.936
sepal_width     2.770
petal_length    4.260
petal_width     1.326
dtype: float64

Inspecting the means it looks like the tree clusters look similar to the actual clusters of setosa, virginica and versicolor as the actual means are close to the final calculated means.